In [5]:
# general libs
import httplib2
import numpy as np
from io import BytesIO
import datetime as dt
# matplotlib plotting
import matplotlib as mpl
from matplotlib import colors
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import matplotlib.animation as animation
# images
import cv2
from PIL import Image
# outlier detection
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
# clustering
from sklearn.cluster import *

In [6]:
url = "http://orm.mipt.ru/archive/port"
port = 47
time = dt.datetime(2016,12,18,20,50)
delta = dt.timedelta(minutes=10)

In [3]:
#download images
h = httplib2.Http('.cache')

for i in range(100000):
        tmp = time - delta*i
        _, content = h.request("http://orm.mipt.ru/archive/port"+ str(port) \
                               + "/" + "{}_{}_{}_{}_{}".format(tmp.year, tmp.month, tmp.day, tmp.hour, tmp.minute) + ".png")
        f = open("img/%d.png" % (i,), "wb")
        f.write(content)
        f.close

In [10]:
# read images

images = []
for i in range(2000):
    j = i
    while True:
        try:
            img = mpimg.imread("img/%d.png" % (j,))
            images.append(img)
        except:
            j = j-1
            continue
        break

In [ ]:
maps = []

colors = {(192,0,192) : 15.0,
         (192,0,128) : 13.0,
         (0,128,0) : 11.0,
         (64,192,0) : 9.5,
         (192,0,0) : 8.5,
         (192,64,0) : 7.5,
         (192,128,0) : 6.5,
         (192,192,0) : 5.5,
         (0,0,128) : 4.5,
         (0,0,192) : 3.5,
         (0,128,192) : 2.5,
         (0,192,192) : 1.5,
         (0,128,128) : 0.65,
         (128,128,128) : 0}

alt = lambda x: colors[(int(x[0]*255), int(x[1]*255), int(x[2]*255))]*255/15

for i in range(len(images)):
    try: 
        maps.append(np.apply_along_axis(alt, 2, images[i][::4,::4,:]).astype(np.uint8))
    except:
        maps.append(np.zeros(maps[0].shape, dtype=np.uint8))

empty = np.zeros(maps[0].shape)
for i in maps:
    empty += i
empty = empty/2000

#fig = plt.figure(figsize=(400,400))
#plt.axis('off')
#plt.imshow(np.apply_along_axis(alt, 2, images[0]), interpolation = "nearest")
#plt.show()

In [21]:
#save video
fig = plt.figure()
plt.axis('off')
ims = [[plt.imshow(i, interpolation="nearest", vmin=0, vmax=255, cmap="Greys")] for i in maps]
ani = animation.ArtistAnimation(fig, ims, interval=10, blit=True,
    repeat_delay=1000)

Writer = animation.writers['ffmpeg']
writer = Writer(fps=30, metadata=dict(artist='Me'), bitrate=5000)
ani.save("flow2000.mp4", writer=writer)

In [27]:
#show animation
fig = plt.figure(figsize=(400,400))
plt.axis('off')
ims = [[plt.imshow((i), interpolation="nearest")] for i in maps]
ani = animation.ArtistAnimation(fig, ims, interval=10, blit=True,
    repeat_delay=1000)

plt.show()

In [ ]:
fig = plt.figure(figsize=(400,400))
plt.axis('off')
plt.imshow(empty, interpolation = "nearest")
plt.show()

In [ ]:
fig = plt.figure(figsize=(400,400))
plt.axis('off')
plt.imshow(maps[0] - (maps[0] != [0,0,0])*128/255, interpolation = "nearest")
plt.show()

In [ ]:
plt.hist(empty[:,:,0].reshape(1,-1)[0], bins = 100, color="r")
plt.hist(empty[:,:,1].reshape(1,-1)[0], bins = 100, color="g")
plt.hist(empty[:,:,2].reshape(1,-1)[0], bins = 100, color="b")
plt.show()